In [1]:
%pip install PyPDF2 python-docx pandas langchain openai pymilvus sentence-transformers scikit-learn python-dotenv


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\latestBranch\Yugm.ai\app\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import PyPDF2
import csv
import pandas as pd
from docx import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')




In [6]:
def extract_text_from_file(file_path):
    """
    Extracts text from a given file based on its extension.

    Args:
        file_path (str): The path to the file.

    Returns:
        str: The extracted text.
    """
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif file_path.endswith('.pdf'):
        # Add PDF extraction logic here
        return "PDF extraction not implemented yet"
    elif file_path.endswith('.docx'):
        # Add DOCX extraction logic here
        return "DOCX extraction not implemented yet"
    else:
        raise ValueError(f"Unsupported file type: {file_path}")


In [7]:
def chunk_document(doc_text, chunk_size=1000, chunk_overlap=200):
    """
    Splits the document text into chunks.

    Args:
        doc_text (str): The document text to be chunked.
        chunk_size (int): The number of characters per chunk.
        chunk_overlap (int): The overlap size between chunks.

    Returns:
        List of text chunks.
    """
    # Assuming RecursiveCharacterTextSplitter is already imported
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return text_splitter.split_text(doc_text)


In [8]:
def vector_database(collection_name, file_path=None, drop_existing_embeddings=False, embeddings_name='sentence',
                    chunk_size=1000, chunk_overlap=200):
    """
    Creates and returns document chunks from the file specified.

    Args:
        file_path: The path to the file for text extraction.
        collection_name: The name of the collection.
        drop_existing_embeddings: Whether to drop existing embeddings.
        embeddings_name: The name of the embeddings ('openai' or 'sentence').
        chunk_size: Maximum number of characters in a chunk.
        chunk_overlap: Number of characters of overlap between chunks.

    Returns:
        List of document chunks.
    """
    # Extract the text from the file
    doc_text = extract_text_from_file(file_path)

    # Chunk the document text before embedding
    doc_chunks = chunk_document(doc_text, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    print(f"Processing file: {file_path}")
    print(f"Total Chunks: {len(doc_chunks)}")
    print("Sample Chunk:")
    print(doc_chunks[0])

    # Here you would typically add logic to store these chunks in a vector database
    return doc_chunks


In [11]:
# Directory containing the files
directory_path = '../data/General/'

# Iterate through the files in the specified directory and process each
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    
    # Ensure the file is supported by checking its extension
    if file_path.endswith(('.txt', '.pdf', '.docx')):
        chunks = vector_database(collection_name="example_collection", file_path=file_path)
        # Here you would typically store the chunks or perform other tasks
    else:
        print(f"Unsupported file type: {filename}")


Processing file: ../data/General/Company K Best Practices.txt
Total Chunks: 6
Sample Chunk:
Company K Best Practices

1. Professional Conduct

1.1 Communication

Clarity: Use clear, concise language in all communications. Avoid jargon or overly complex terms unless necessary.
Respect: Be courteous and professional in all interactions. Listen actively and respond thoughtfully.
Timeliness: Respond to emails and messages promptly, ideally within 24 hours.
1.2 Time Management

Prioritization: Prioritize tasks based on urgency and importance. Use tools like to-do lists or project management software to stay organized.
Deadlines: Meet deadlines consistently. If delays are anticipated, inform relevant stakeholders as early as possible.
2. Workplace Behavior

2.1 Team Collaboration

Teamwork: Collaborate effectively with colleagues. Share knowledge and resources to achieve common goals.
Feedback: Provide constructive feedback and be open to receiving it. Use feedback as a tool for personal and

In [4]:
%pip install sentence-transformers scikit-learn torch


You should consider upgrading via the 'e:\latestBranch\Yugm.ai\app\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
%pip uninstall torch transformers sentence-transformers





^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Note: you may need to restart the kernel to use updated packages.Looking in indexes: https://download.pytorch.org/whl/cpu



You should consider upgrading via the 'e:\latestBranch\Yugm.ai\app\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
%pip install sentence-transformers

You should consider upgrading via the 'e:\latestBranch\Yugm.ai\app\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [15]:
%pip install transformers torch


You should consider upgrading via the 'e:\latestBranch\Yugm.ai\app\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from data import load_n_split  # Assuming your script is named data.py

# Load the BAAI model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en")
model = AutoModel.from_pretrained("BAAI/bge-large-en")

def get_embeddings(texts):
    """
    Generate embeddings for the given texts using BAAI model.

    Args:
        texts (list): List of texts for which embeddings are to be generated.

    Returns:
        numpy.ndarray: Array of embeddings.
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # We use the pooled output (which represents [CLS] token) as the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def rank_chunks(chunks, query):
    """
    Rank the chunks based on similarity to the query using the BAAI model.

    Args:
        chunks (list): List of document chunks.
        query (str): The query for which to rank the chunks.

    Returns:
        List of tuples (chunk, score) sorted by relevance.
    """
    # Get embeddings for the chunks and the query
    chunk_texts = [chunk.page_content for chunk in chunks]
    chunk_embeddings = get_embeddings(chunk_texts)
    query_embedding = get_embeddings([query])

    # Compute cosine similarity between query and each chunk
    similarity_scores = cosine_similarity(query_embedding, chunk_embeddings)[0]

    # Pair each chunk with its similarity score
    ranked_chunks = [(chunk, score) for chunk, score in zip(chunks, similarity_scores)]

    # Sort the chunks by their score in descending order (most relevant first)
    ranked_chunks = sorted(ranked_chunks, key=lambda x: x[1], reverse=True)

    return ranked_chunks

# Load the chunks from the directory (same as the previous code block)
directory_path = '../data/Organizational Information/'
chunks = load_n_split(path=directory_path, splitter="token")

# Example query
query = "what should i do to enhance my skills?"

# Rank the chunks based on the query
ranked_chunks = rank_chunks(chunks, query)

# Print the ranked chunks and their scores
for i, (chunk, score) in enumerate(ranked_chunks):
    print(f"Rank {i+1} (Score: {score}):\n{chunk.page_content}\n")


Rank 1 (Score: 0.7559425830841064):
ities - Encouragement: Employees are actively encouraged to pursue new challenges and roles within the company to enhance their skills and advance their career trajectory. - Impact: This focus on internal mobility promotes a more engaged workforce and aligns individual aspirations with organizational goals. - Mentorship Programs - Support System: Internal candidates who transition into new roles or departments may be paired with a mentor to facilitate their adjustment and provide guidance. - Benefit: Mentorship helps employees acclimate to their new positions and fosters professional development. 5. Benefits and Impact The Job Openings and Internal Transfers policy offers several advantages for both employees and the organization:- Enhanced Employee Retention: By providing growth opportunities internally, Company K cultivates loyalty among its employees and reduces turnover rates. - Improved Employee Engagement: A clear pathway for advancement encour

In [19]:
%pip install rank_bm25


You should consider upgrading via the 'e:\latestBranch\Yugm.ai\app\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [21]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from data import load_n_split  # Assuming your script is named data.py

# Load the BAAI model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en")
model = AutoModel.from_pretrained("BAAI/bge-large-en")

def get_embeddings(texts):
    """
    Generate embeddings for the given texts using BAAI model.

    Args:
        texts (list): List of texts for which embeddings are to be generated.

    Returns:
        numpy.ndarray: Array of embeddings.
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # We use the pooled output (which represents [CLS] token) as the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def rank_chunks_with_bm25(chunks, query):
    """
    First rank chunks based on cosine similarity to the query using the BAAI model.
    Then, rerank the top chunks using BM25.

    Args:
        chunks (list): List of document chunks.
        query (str): The query for which to rank the chunks.

    Returns:
        List of tuples (chunk, score) sorted by BM25 relevance.
    """
    # Get the chunk texts
    chunk_texts = [chunk.page_content for chunk in chunks]

    # Rank the chunks using embeddings
    chunk_embeddings = get_embeddings(chunk_texts)
    query_embedding = get_embeddings([query])
    
    # Compute cosine similarity between query and each chunk
    similarity_scores = cosine_similarity(query_embedding, chunk_embeddings)[0]

    # Pair each chunk with its similarity score
    ranked_chunks = [(chunk, score) for chunk, score in zip(chunks, similarity_scores)]

    # Sort the chunks by their cosine similarity score in descending order (most relevant first)
    ranked_chunks = sorted(ranked_chunks, key=lambda x: x[1], reverse=True)

    # After initial ranking with cosine similarity, rerank using BM25
    bm25_corpus = [chunk.page_content.split() for chunk, _ in ranked_chunks]  # Tokenize chunks
    bm25 = BM25Okapi(bm25_corpus)

    # Tokenize the query for BM25
    query_tokens = query.split()

    # Get BM25 scores for the query
    bm25_scores = bm25.get_scores(query_tokens)

    # Combine BM25 scores with ranked chunks
    reranked_chunks = [(chunk, bm25_score) for (chunk, _), bm25_score in zip(ranked_chunks, bm25_scores)]

    # Sort again by BM25 score in descending order
    reranked_chunks = sorted(reranked_chunks, key=lambda x: x[1], reverse=True)

    return reranked_chunks

# Load the chunks from the directory (same as the previous code block)
directory_path = '../data/Organizational Information/'
chunks = load_n_split(path=directory_path, splitter="token")

# Example query
query = "i want to knoe if there are any internal opening for me?"

# Rank the chunks based on the query, and then rerank using BM25
reranked_chunks = rank_chunks_with_bm25(chunks, query)

# Print the ranked chunks and their BM25 scores
for i, (chunk, score) in enumerate(reranked_chunks):
    print(f"Rank {i+1} (BM25 Score: {score}):\n{chunk.page_content}\n")


Rank 1 (BM25 Score: 5.130289239305884):
 is always accurate. - This ensures that any changes to leadership or team composition are visible to everyone in the company without delay. - New Hires - Newly hired employees are automatically added to the org chart within their respective departments as soon as they complete their onboarding process. This ensures that their position and reporting lines are clear from day one. 4. Use Cases The Company Org Chart is a vital tool for several practical applications within Company K, providing benefits to both new and existing employees. - New Employee Orientation - The org chart serves as an essential resource for new employees, helping them quickly understand the company’s structure and learn whom to contact within different departments. - It provides clarity on reporting lines, enabling new hires to know where they fit into the organizational hierarchy from the start. - Project Assignments - Managers can use the org chart to determine reporting l